# PYDANTIC
- framework for defining specialized Python classes
- class attributes are called `fields`
- the specialized class is called a `model`
- simple way to `load` data from a dict or JSON into a model
- simple way to `extract` model instance data to dict or JSON
- Also provides a robust system to validate data during deserialization
- enables us to work with our data in `OOP` manner

### Creating a Pydantic Model

In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [3]:
p = Person(first_name='Guido', last_name='Rossum', age=42)
p

Person(first_name='Guido', last_name='Rossum', age=42)

In [4]:
str(p)

"first_name='Guido' last_name='Rossum' age=42"

In [5]:
repr(p)

"Person(first_name='Guido', last_name='Rossum', age=42)"

In [6]:
p.model_fields

/var/folders/lp/7jt9d_5x7g106b7mc39v5nlh0000gn/T/ipykernel_5418/1166466245.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  p.model_fields


{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True)}

In [7]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=True)}

In [9]:
from pydantic import ValidationError

In [10]:
try:
    Person(last_name='Rossum')
except ValidationError as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'last_name': 'Rossum'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
age
  Field required [type=missing, input_value={'last_name': 'Rossum'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


In [11]:
try:
    Person(last_name='Rossum')
except Exception as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'last_name': 'Rossum'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
age
  Field required [type=missing, input_value={'last_name': 'Rossum'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


In [12]:
p.age = 24
p

Person(first_name='Guido', last_name='Rossum', age=24)

### By default, Pydantic automatically converts compatible types to match your field annotations.

In [20]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    
try:
    p1 = Person(first_name='Guido', last_name='Rossum', age='42')
except ValidationError as ex:
    print(ex)

In [24]:
isinstance(p1.age, int)

True

In [25]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int
    
try:
    p1 = Person(first_name='Guido', last_name='Rossum', age='forty-two')
except ValidationError as ex:
    print(ex)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='forty-two', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


In [26]:
p1

Person(first_name='Guido', last_name='Rossum', age=42)

### once created pydantic wont stop us from changing the data. Validation happens when we load the data to a pydantic model 

In [27]:
p1.age = 'forty-two'

In [28]:
p1

Person(first_name='Guido', last_name='Rossum', age='forty-two')

### Deserialization

In [29]:
from pydantic import BaseModel, ValidationError

In [30]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [31]:
data = {
    "first_name": "Guido",
    "last_name": "Rossum",
    "age": 42
}

In [33]:
p = Person.model_validate(data)
p

Person(first_name='Guido', last_name='Rossum', age=42)

In [37]:
data_json= '''{"first_name": "Isaac","last_name":"newton", "age":52}'''

p = Person.model_validate_json(data_json)
p

Person(first_name='Isaac', last_name='newton', age=52)

### catches invalid json as well as model incompatible issues

In [38]:
data_json= '''{"first_name": "Isaac","last_name":"newton", "age":52,}'''

p = Person.model_validate_json(data_json)
p

ValidationError: 1 validation error for Person
  Invalid JSON: trailing comma at line 1 column 55 [type=json_invalid, input_value='{"first_name": "Isaac","...e":"newton", "age":52,}', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/json_invalid

In [40]:
data_json= '''{"first_name": "Isaac","last_name":"newton", "age":"fifty"}'''

p = Person.model_validate_json(data_json)
p

ValidationError: 1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='fifty', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing

### Serialization

In [41]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

p1 = Person(first_name='Galelio', last_name='Galei', age=42)

p1.__dict__

{'first_name': 'Galelio', 'last_name': 'Galei', 'age': 42}

In [42]:
p1.model_dump()

{'first_name': 'Galelio', 'last_name': 'Galei', 'age': 42}

In [44]:
type(p1.model_dump())

dict

In [43]:
p1.model_dump_json()

'{"first_name":"Galelio","last_name":"Galei","age":42}'

In [45]:
type(p1.model_dump_json())

str

In [46]:
p1.model_dump(exclude=['first_name'])

{'last_name': 'Galei', 'age': 42}

In [47]:
p1.model_dump_json(exclude=['age', 'first_name'])

'{"last_name":"Galei"}'

In [48]:
p1.model_dump_json(include=['age'])

'{"age":42}'

In [49]:
p1.model_dump(include=['first_name'])

{'first_name': 'Galelio'}

### Required vs Optional Fields

In [50]:
class Circle(BaseModel):
    center: tuple[int, int]
    radius: int=1

In [53]:
Circle.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=True),
 'radius': FieldInfo(annotation=int, required=False, default=1)}

In [58]:
c1 = Circle(center=(1,1), radius=1)
c1

Circle(center=(1, 1), radius=1)

In [59]:
c1 = Circle(center=(0.1,1.5), radius=1)
c1

ValidationError: 2 validation errors for Circle
center.0
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=0.1, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/int_from_float
center.1
  Input should be a valid integer, got a number with a fractional part [type=int_from_float, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/int_from_float

### Nullable fields

In [60]:
class MyModel(BaseModel):
    field: int

try:
    MyModel(field=None)
except ValidationError as ex:
    print(ex)

1 validation error for MyModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type


In [62]:
class MyModel(BaseModel):
    field: int

try:
    MyModel()
except ValidationError as ex:
    print(ex)

1 validation error for MyModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


In [63]:
class MyModel(BaseModel):
    field: int | None

try:
    MyModel(field=None)
    print('Model created')
except ValidationError as ex:
    print(ex)

Model created


In [64]:
class MyModel(BaseModel):
    field: int | None

try:
    MyModel()
    print('Model created')
except ValidationError as ex:
    print(ex)

1 validation error for MyModel
field
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing


In [66]:
class MyModel(BaseModel):
    field: int | None = None   # another way union[int, None] = None or Optional[int]

try:
    MyModel()
    print('Model created')
except ValidationError as ex:
    print(ex)

Model created


### to make a field nullable provide a default value as Null and also add Null to field.nay field without a default value is Required !

In [68]:
MyModel.model_fields

{'field': FieldInfo(annotation=Union[int, NoneType], required=False, default=None)}

### Required by not Nullable - below is a common mistake

In [76]:
class MyModel(BaseModel):
    field: int = None

try:
    m = MyModel()
    print(m)
except ValidationError as ex:
    print(ex)

field=None


### Remember pydantic do not validate the fields with defaults specified! it trusts us to provide valid defaults and hence no checks but see this now. 

In [77]:
try:
    m = MyModel(field=None)
    print(m)
except ValidationError as ex:
    print(ex)

1 validation error for MyModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type


In [78]:
class MyModel(BaseModel):
    field: int | None = None

try:
    m = MyModel()
    print(m)
except ValidationError as ex:
    print(ex)

field=None


In [79]:
try:
    m = MyModel(field=None)
    print(m)
except ValidationError as ex:
    print(ex)

field=None


### Optional, Not nullable

In [82]:
class MyModel(BaseModel):
    field: int = 0

try:
    m = MyModel()
    print(m)
except ValidationError as ex:
    print(ex)

###############

try:
    m = MyModel(field=None)
    print(m)
except ValidationError as ex:
    print(ex)

field=0
1 validation error for MyModel
field
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type


### inspecting fields

In [89]:
class Circle(BaseModel):
    center_x: int = 0
    center_y: int = 0
    radius: float
    name: str | None = None

c1 = Circle(radius = 12, name = "my_circle")
c1

Circle(center_x=0, center_y=0, radius=12.0, name='my_circle')

In [90]:
c1.model_fields

/var/folders/lp/7jt9d_5x7g106b7mc39v5nlh0000gn/T/ipykernel_5418/1104979828.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  c1.model_fields


{'center_x': FieldInfo(annotation=int, required=False, default=0),
 'center_y': FieldInfo(annotation=int, required=False, default=0),
 'radius': FieldInfo(annotation=float, required=True),
 'name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

In [91]:
c1.model_fields_set

{'name', 'radius'}

### to generate only the fields set by the model 

In [92]:
c1.model_dump(include=c1.model_fields_set)

{'radius': 12.0, 'name': 'my_circle'}

### JSON Schema Generator

In [93]:
class MyModel(BaseModel):
    field_1: int | None = None
    field_2: str = "Python"

MyModel.model_json_schema()

{'properties': {'field_1': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'default': None,
   'title': 'Field 1'},
  'field_2': {'default': 'Python', 'title': 'Field 2', 'type': 'string'}},
 'title': 'MyModel',
 'type': 'object'}

In [94]:
from pprint import pprint

In [95]:
pprint(MyModel.model_json_schema())

{'properties': {'field_1': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
                            'default': None,
                            'title': 'Field 1'},
                'field_2': {'default': 'Python',
                            'title': 'Field 2',
                            'type': 'string'}},
 'title': 'MyModel',
 'type': 'object'}


# MODEL CONFIGURATION

### EXTRA FIELDS

In [96]:
class Model(BaseModel):
    field_1: int

In [97]:
m = Model(field_1 = 10, field_2 = 20)
m

Model(field_1=10)

In [98]:
m.__dict__

{'field_1': 10}

### by default pydantic ignores any extra fields unless set by a config

In [99]:
from pydantic import ConfigDict

In [104]:
class Model(BaseModel):
    model_config = ConfigDict(extra='allow')
    
    field_1: int

m = Model(field_1=10, filed_2=20, field_3=30)
m

Model(field_1=10, filed_2=20, field_3=30)

In [105]:
m.model_extra

{'filed_2': 20, 'field_3': 30}

In [106]:
m.model_fields

/var/folders/lp/7jt9d_5x7g106b7mc39v5nlh0000gn/T/ipykernel_5418/3529051558.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  m.model_fields


{'field_1': FieldInfo(annotation=int, required=True)}

In [107]:
m.__dict__

{'field_1': 10}

In [108]:
m.model_fields_set

{'field_1', 'field_3', 'filed_2'}

In [109]:
class Model(BaseModel):
    model_config = ConfigDict(extra='forbid')
    
    field_1: int

m = Model(field_1=10, filed_2=20)
m

ValidationError: 1 validation error for Model
filed_2
  Extra inputs are not permitted [type=extra_forbidden, input_value=20, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/extra_forbidden

### STRICT and LAX Type Coercion

In [111]:
class MyModel(BaseModel):
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

try:
    MyModel(field_1=100, field_2=10, field_3=(1, 2, 3), field_4=[21, 22, 23])
except ValidationError as ex:
    print(ex)

1 validation error for MyModel
field_1
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type


In [114]:
class MyModel(BaseModel):
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

try:
    m = MyModel(field_1='abc', field_2=10, field_3=(1, 2, 3), field_4=[21, 22, 23])
    print(m)
except ValidationError as ex:
    print(ex)

field_1='abc' field_2=10.0 field_3=[1, 2, 3] field_4=(21, 22, 23)


In [121]:
json_data = '''
{
    "field_01": true,
    "field_02": 10.5,
    "field_03": 10,
    "field_04": null,
    "field_05": [1, 2, 3],
    "field_06": {
        "a": 1,
        "b": 2, 
        "c": [3, 4, 5]
    },
    "field_07":[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    ]
}
'''

In [122]:
import json
from pprint import pprint

data = json.loads(json_data)
pprint(data)

{'field_01': True,
 'field_02': 10.5,
 'field_03': 10,
 'field_04': None,
 'field_05': [1, 2, 3],
 'field_06': {'a': 1, 'b': 2, 'c': [3, 4, 5]},
 'field_07': [[1, 0, 0], [0, 1, 0], [0, 0, 1]]}


### all arrays got coerced into lists 
- may be we wanted them as tuples,
- may be instead of all objects as dicts we wanted them as other classes.
- may be we wanted floats and not ints

### Strict mode behavior can be controlled at the configuration level. When used on a Pydantic model (or model like class such as dataclasses), strictness can still be overridden at the field level:

In [124]:
class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

try:
    m = MyModel(field_1='abc', field_2=10, field_3=(1, 2, 3), field_4=[21, 22, 23])
    print(m)
except ValidationError as ex:
    print(ex)

2 validation errors for MyModel
field_3
  Input should be a valid list [type=list_type, input_value=(1, 2, 3), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.12/v/list_type
field_4
  Input should be a valid tuple [type=tuple_type, input_value=[21, 22, 23], input_type=list]
    For further information visit https://errors.pydantic.dev/2.12/v/tuple_type


In [125]:
class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_1: str
    field_2: float
    field_3: list
    field_4: tuple

try:
    m = MyModel(field_1='abc', field_2=10, field_3=[1, 2, 3], field_4=(21, 22, 23))
    print(m)
except ValidationError as ex:
    print(ex)

field_1='abc' field_2=10.0 field_3=[1, 2, 3] field_4=(21, 22, 23)


In [129]:
json_data = '''
{
    "field_01": true,
    "field_02": 10.5,
    "field_03": 10,
    "field_04": null,
    "field_05": [1, 2, 3],
    "field_06": {
        "a": 1,
        "b": 2, 
        "c": [3, 4, 5]
    },
    "field_07":[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    ]
}
'''

class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_01: bool
    field_02: float
    field_03: float
    field_04: str | None = None
    field_05: tuple[int, ...]
    field_06: dict
    field_07: list[list, ...]

m = MyModel.model_validate_json(json_data)
m

MyModel(field_01=True, field_02=10.5, field_03=10.0, field_04=None, field_05=(1, 2, 3), field_06={'a': 1, 'b': 2, 'c': [3, 4, 5]}, field_07=[[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [130]:
type(m.field_05)

tuple

In [135]:
json_data = '''
{
    "field_01": true,
    "field_02": 10.5,
    "field_03": 10,
    "field_04": null,
    "field_05": [1, 2, 3.4],
    "field_06": {
        "a": 1,
        "b": 2, 
        "c": [3, 4, 5.1]
    },
    "field_07":[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1.0]
    ]
}
'''

class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_01: bool
    field_02: float
    field_03: float
    field_04: str | None = None
    field_05: tuple[int, ...]
    field_06: dict
    field_07: list[list[int,...], ...]

m = MyModel.model_validate_json(json_data)
m

ValidationError: 2 validation errors for MyModel
field_05.2
  Input should be a valid integer [type=int_type, input_value=3.4, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type
field_07.2.2
  Input should be a valid integer [type=int_type, input_value=1.0, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type

In [138]:
json_data = '''
{
    "field_01": true,
    "field_02": 10.5,
    "field_03": 10,
    "field_04": null,
    "field_05": [1, 2, 3.4],
    "field_06": {
        "a": 1,
        "b": 2, 
        "c": [3, 4, 5.1]
    },
    "field_07":[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1.0]
    ]
}
'''

class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_01: bool
    field_02: float
    field_03: float
    field_04: str | None = None
    field_05: tuple[int, ...]
    field_06: dict[Any, int]
    field_07: list[list[int,...], ...]

m = MyModel.model_validate_json(json_data)
m

NameError: name 'Any' is not defined

In [144]:
from typing import Any

json_data = '''
{
    "field_01": true,
    "field_02": 10.5,
    "field_03": 10,
    "field_04": null,
    "field_05": [1, 2, 3],
    "field_06": {
        "a": 1,
        "b": 2, 
        "c": [3, 4, 5]
    },
    "field_07":[
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
    ]
}
'''

class MyModel(BaseModel):
    model_config = ConfigDict(strict=True, extra='forbid')
    
    field_01: bool
    field_02: float
    field_03: float
    field_04: str | None = None
    field_05: tuple[int, ...]
    field_06: dict[Any, int|list]
    field_07: list[list[int,...]] 

m = MyModel.model_validate_json(json_data)
m

MyModel(field_01=True, field_02=10.5, field_03=10.0, field_04=None, field_05=(1, 2, 3), field_06={'a': 1, 'b': 2, 'c': [3, 4, 5]}, field_07=[[1, 0, 0], [0, 1, 0], [0, 0, 1]])

## Validating Default Values

In [146]:
class MyModel(BaseModel):
    field_1: int = None
    field_2: str = 100

MyModel()

MyModel(field_1=None, field_2=100)

### python happily allows above without any checks but will validate any user passed args

In [147]:
MyModel(field_1= None, field_2=100)

ValidationError: 2 validation errors for MyModel
field_1
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type
field_2
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

### we can enforce both model level and field level validator for default values too

In [148]:
class MyModel(BaseModel):
    model_config = ConfigDict(validate_default=True, strict=True, extra='forbid')
    
    field_1: int = None
    field_2: str = 100

MyModel()

ValidationError: 2 validation errors for MyModel
field_1
  Input should be a valid integer [type=int_type, input_value=None, input_type=NoneType]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type
field_2
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

In [151]:
class MyModel(BaseModel):
    model_config = ConfigDict(validate_default=True, strict=True, extra='forbid')
    
    field_1: int = 1
    field_2: str = "100"

MyModel()

MyModel(field_1=1, field_2='100')

In [154]:
MyModel(field_1=1, field_2="1")

MyModel(field_1=1, field_2='1')

## Validating Assignments

In [157]:
class MyModel(BaseModel):
    model_config = ConfigDict(validate_default=True, strict=True, extra='forbid')
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [158]:
m.field_1 = 'python'
m

MyModel(field_1='python', field_2='100')

### python does not validate the model once created ! 

In [165]:
class MyModel(BaseModel):
    model_config = ConfigDict(validate_assignment=True, 
                              validate_default=True, 
                              strict=True, 
                              extra='forbid')
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [166]:
m.field_1 = 'python'
m

ValidationError: 1 validation error for MyModel
field_1
  Input should be a valid integer [type=int_type, input_value='python', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type

## Mutability

In [167]:
class MyModel(BaseModel):
    model_config = ConfigDict(validate_assignment=True, 
                              validate_default=True, 
                              strict=True, 
                              extra='forbid')
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [168]:
m.field_2 = 'python'
m

MyModel(field_1=1, field_2='python')

In [171]:
class MyModel(BaseModel):
    model_config = ConfigDict(frozen=True, # prevents mutations
                              validate_assignment=True, # validate assigments too
                              validate_default=True, # validate default values
                              strict=True, # to typecasting 
                              extra='forbid') # no extra fields other than specified in model
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [172]:
m.field_2 = 'python'

ValidationError: 1 validation error for MyModel
field_2
  Instance is frozen [type=frozen_instance, input_value='python', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/frozen_instance

## Coercing Numbers to Strings

In [179]:
class MyModel(BaseModel):
    model_config = ConfigDict(#frozen=True, # prevents mutations
                              validate_assignment=True, # validate assigments too
                              validate_default=True, # validate default values
                              strict=True, # to typecasting 
                              extra='forbid', # no extra fields other than specified in model
                              coerce_numbers_to_str=True  # works only in lax mode not strict !
                             ) 
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [180]:
m.field_1 = "2"

ValidationError: 1 validation error for MyModel
field_1
  Input should be a valid integer [type=int_type, input_value='2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_type

In [177]:
class MyModel(BaseModel):
    model_config = ConfigDict(
                            # frozen=True, # prevents mutations
                              validate_assignment=True, # validate assigments too
                              validate_default=True, # validate default values
                            # strict=True, # to typecasting 
                              extra='forbid', # no extra fields other than specified in model
                              coerce_numbers_to_str=True  # works only in lax mode not strict !
                             ) 
    
    field_1: int = 1
    field_2: str = "100"

m = MyModel()
m

MyModel(field_1=1, field_2='100')

In [178]:
m.field_1 = "2"
m

MyModel(field_1=2, field_2='100')

## Standardizing Strings

In [181]:
m1 = '   python  '
m2 = 'python \t \n'
m1 == m2, m1.strip() == m2.strip()

(False, True)

In [182]:
class MyModel(BaseModel):
    model_config = ConfigDict()

    field_01: str

m1 = MyModel(field_01='   python  ')
m2 = MyModel(field_01='python \t \n')

In [183]:
m1, m2

(MyModel(field_01='   python  '), MyModel(field_01='python \t \n'))

In [184]:
m1 == m2

False

In [187]:
class MyModel(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True)

    field_01: str

m1 = MyModel(field_01='   python  ')
m2 = MyModel(field_01='python \t \n')

In [188]:
m1, m2

(MyModel(field_01='python'), MyModel(field_01='python'))

In [189]:
m1 == m2

True

In [191]:
class MyModel(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True,
                             str_to_lower=True)

    field_01: str

m1 = MyModel(field_01='   PytHoN  ')
m2 = MyModel(field_01='pYThon \t \n')

m1, m2

(MyModel(field_01='python'), MyModel(field_01='python'))

In [192]:
class MyModel(BaseModel):
    model_config = ConfigDict(str_strip_whitespace=True,
                             str_to_upper=True)

    field_01: str

m1 = MyModel(field_01='   PytHoN  ')
m2 = MyModel(field_01='pYThon \t \n')

m1, m2

(MyModel(field_01='PYTHON'), MyModel(field_01='PYTHON'))

## ENUMS in pydantic

In [193]:
from enum import Enum

In [208]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

Colour.RED

<Colour.RED: 'red'>

In [222]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class MyModel(BaseModel):
    model_config = ConfigDict(use_enum_values=True)
    
    colour: Colour

In [223]:
m = MyModel(colour = Colour.RED)
m

MyModel(colour='red')

In [224]:
type(m.colour)

str

In [225]:
# serializing a pydantic model would retain 
m.model_dump(), m.model_dump_json()

({'colour': 'red'}, '{"colour":"red"}')

In [227]:
m.model_dump()

{'colour': 'red'}

In [218]:
# deserializing

json_data = '''
{
    "colour": "red"
}
'''

m = MyModel.model_validate_json(json_data)
m

MyModel(colour='red')

In [220]:
# see the error ! Pydantic validates the enum
json_data = '''
{
    "colour": "pink"
}
'''

m = MyModel.model_validate_json(json_data)
m

ValidationError: 1 validation error for MyModel
colour
  Input should be 'red', 'blue' or 'green' [type=enum, input_value='pink', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/enum

### `ConfigDict(use_enum_values=True)` : “Whenever you see an Enum — during validation, assignment, or model dumping — automatically convert it to its .value instead of keeping the Enum object.”

In [231]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class MyModel(BaseModel):
    model_config = ConfigDict()  # NOTE NO CONFIG PROVIDED
    
    colour: Colour

MyModel(colour=Colour.PINK)

AttributeError: type object 'Colour' has no attribute 'PINK'

In [233]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class MyModel(BaseModel):
    model_config = ConfigDict()  # NOTE NO CONFIG PROVIDED
    
    colour: Colour

m = MyModel(colour=Colour.RED)
m

MyModel(colour=<Colour.RED: 'red'>)

In [235]:
m.model_dump()

{'colour': <Colour.RED: 'red'>}

In [237]:
type(m.colour)

<enum 'Colour'>

## default values and caveats

In [242]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class MyModel(BaseModel):
    model_config = ConfigDict(use_enum_values=True)  
    
    colour: Colour = Colour.BLUE

m = MyModel()
m

MyModel(colour=<Colour.BLUE: 'blue'>)

In [243]:
m.model_dump()

{'colour': <Colour.BLUE: 'blue'>}

## by default pydantic does not do default checks unless asked so 

In [244]:
class Colour(Enum):
    RED = 'red'
    BLUE = 'blue'
    GREEN = 'green'

class MyModel(BaseModel):
    model_config = ConfigDict(use_enum_values=True,
                             validate_default=True)  
    
    colour: Colour = Colour.BLUE

m = MyModel()
m

MyModel(colour='blue')

In [246]:
m.model_dump()

{'colour': 'blue'}

## FIELD AILASING

In [285]:
from pydantic import BaseModel, Field, ValidationError

In [286]:
class Model(BaseModel):
    id_: int = Field(alias='id')
    last_name: str = Field(alias='lastName')

In [291]:
json_data = '''
{
    "id": 100,
    "lastName": "Gauss"
}
'''

In [292]:
m = Model.model_validate_json(json_data)
m

Model(id_=100, last_name='Gauss')

## De-serializaiton uses alias and model uses our fields

In [295]:
m = Model(id=100, lastName='Einstein')
m

Model(id_=100, last_name='Einstein')

## inside the model we still use field names

In [299]:
m.id_, m.last_name

(100, 'Einstein')

## we can add defaults

In [300]:
class Model(BaseModel):
    id_: int = Field(alias='id', default=42)
    last_name: str = Field(alias='lastName', default='fibonacchi')

m = Model()
m

Model(id_=42, last_name='fibonacchi')

In [302]:
m.model_dump_json(), m.model_dump()

('{"id_":42,"last_name":"fibonacchi"}', {'id_': 42, 'last_name': 'fibonacchi'})

## when we define alias - the only thing that gets affected is - while serializing

In [303]:
class Person(BaseModel):
    id_: int = Field(alias='id')
    first_name: str = Field(alias='firstName')
    last_name: str | None = Field(alias='lastName', default=None)
    age: int | None = None

isaac = Person(id=1, firstName='Isaac', age=84)
isaac

Person(id_=1, first_name='Isaac', last_name=None, age=84)

In [304]:
isaac.model_dump()

{'id_': 1, 'first_name': 'Isaac', 'last_name': None, 'age': 84}

In [305]:
isaac.model_dump(by_alias=True)

{'id': 1, 'firstName': 'Isaac', 'lastName': None, 'age': 84}

In [306]:
isaac.model_dump_json(by_alias=True)

'{"id":1,"firstName":"Isaac","lastName":null,"age":84}'

In [307]:
isaac.model_fields

/var/folders/lp/7jt9d_5x7g106b7mc39v5nlh0000gn/T/ipykernel_5418/4274488072.py:1: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  isaac.model_fields


{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'first_name': FieldInfo(annotation=str, required=True, alias='firstName', alias_priority=2),
 'last_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='lastName', alias_priority=2),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, default=None)}

## Alias Generator Functions

In [308]:
from pydantic.alias_generators import to_camel, to_snake, to_pascal

In [309]:
to_camel('last_name')

'lastName'

In [311]:
to_snake('firstName')

'first_name'

In [312]:
to_pascal('last_name')

'LastName'

In [313]:
def make_upper(in_str: str) -> str:
    return in_str.upper()

In [314]:
make_upper('randomName')

'RANDOMNAME'

In [319]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)
    
    id_: int
    first_name: str 
    last_name: str | None = None
    age: int | None = None

p = Person(ID_=1, FIRST_NAME='Piere', LAST_NAME='Pascal', AGE=42)
p

Person(id_=1, first_name='Piere', last_name='Pascal', age=42)

In [317]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='ID_', alias_priority=1),
 'first_name': FieldInfo(annotation=str, required=True, alias='FIRST_NAME', alias_priority=1),
 'last_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='LAST_NAME', alias_priority=1),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, default=None, alias='AGE', alias_priority=1)}

In [320]:
p.model_dump(), p.model_dump(by_alias=True)

({'id_': 1, 'first_name': 'Piere', 'last_name': 'Pascal', 'age': 42},
 {'ID_': 1, 'FIRST_NAME': 'Piere', 'LAST_NAME': 'Pascal', 'AGE': 42})

## override model alias with field level alias

In [322]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)
    
    id_: int = Field(alias='ID')
    first_name: str 
    last_name: str | None = None
    age: int | None = None

p = Person(ID=1, FIRST_NAME='Piere', LAST_NAME='Pascal', AGE=42)
p

Person(id_=1, first_name='Piere', last_name='Pascal', age=42)

In [323]:
p.model_dump(), p.model_dump(by_alias=True)

({'id_': 1, 'first_name': 'Piere', 'last_name': 'Pascal', 'age': 42},
 {'ID': 1, 'FIRST_NAME': 'Piere', 'LAST_NAME': 'Pascal', 'AGE': 42})

In [326]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel)
    
    id_: int = Field(alias='id')
    first_name: str 
    last_name: str | None = None
    age: int | None = None

Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'first_name': FieldInfo(annotation=str, required=True, alias='firstName', alias_priority=1),
 'last_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None, alias='lastName', alias_priority=1),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, default=None, alias='age', alias_priority=1)}

## why do we need alias ? 
- API which gives data might use python `reserve words` like `list`, `id`, we would want to control that

In [328]:
p = Person(id=1, firstName='luis', lastName='Fourier', age=62)
p

Person(id_=1, first_name='luis', last_name='Fourier', age=62)

In [330]:
p = Person(id=1, firstName='luis', lastName='Fourier', age=62)
p

Person(id_=1, first_name='luis', last_name='Fourier', age=62)

In [333]:
p.model_dump(by_alias=True), p.model_dump()

({'id': 1, 'firstName': 'luis', 'lastName': 'Fourier', 'age': 62},
 {'id_': 1, 'first_name': 'luis', 'last_name': 'Fourier', 'age': 62})

In [331]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel)
    
    id_: int = Field(alias='id')
    first_name: str 
    last_name: str | None = None
    age: int | None = None

p = Person(id_=1, firstName='luis', last_name='Fourier', age=62)
p

ValidationError: 1 validation error for Person
id
  Field required [type=missing, input_value={'id_': 1, 'firstName': '...': 'Fourier', 'age': 62}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

## PROJECT

you should create an Automobile model that contains the following fields:

- manufacturer, string, required, not nullable
- series_name, string, required, not nullable
- type_, string, required, not nullable
- is_electric, boolean, defaults to False, not nullable
- manufactured_date, date, required (hint use date from datetime module as your - field type hint), not nullable
- base_msrp_usd, float, required, not nullable
- vin, string, required, not nullable
- number_of_doors, integer, defaults to 4, not nullable
- registration_country, string, defaults to None
- license_plate, string, defaults to None

In [252]:
from datetime import date

In [254]:
class Automobile(BaseModel):
    manufacturer: str
    series_name: str
    type_: str
    is_electric: bool = False
    manufactured_date: date
    base_msrp_usd: float
    vin: str
    number_of_doors: int = 4
    registration_country: str | None = None
    license_plate: str | None = None

Automobile()

ValidationError: 6 validation errors for Automobile
manufacturer
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
series_name
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
type_
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
manufactured_date
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
base_msrp_usd
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing
vin
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/missing

In [268]:
class Automobile(BaseModel):
    model_config = ConfigDict(extra='forbid')
                              
    manufacturer: str
    series_name: str
    type_: str
    is_electric: bool = False
    manufactured_date: date
    base_msrp_usd: float
    vin: str
    number_of_doors: int = 4
    registration_country: str | None = None
    license_plate: str | None = None

Automobile(manufacturer='volvo', 
           series_name='v60', 
           type_='electric', 
           manufactured_date=date(2025, 12, 12), 
           base_msrp_usd=499.99, 
           vin='abc123')

Automobile(manufacturer='volvo', series_name='v60', type_='electric', is_electric=False, manufactured_date=datetime.date(2025, 12, 12), base_msrp_usd=499.99, vin='abc123', number_of_doors=4, registration_country=None, license_plate=None)

In [269]:
# Python dictionary
from datetime import date

data = {
    "manufacturer": "BMW",
    "series_name": "M4",
    "type_": "Convertible",
    "is_electric": False,
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93_300,
    "vin": "1234567890",
    "number_of_doors": 2,
    "registration_country": "France",
    "license_plate": "AAA-BBB",
}

data_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': 'Convertible',
    'is_electric': False,
    'manufactured_date': date(2023,1,1),
    'base_msrp_usd': 93_300,
    'vin': '1234567890',
    'number_of_doors': 2,
    'registration_country': 'France',
    'license_plate': 'AAA-BBB',
}


# JSON
data_json = '''
{
    "manufacturer": "BMW",
    "series_name": "M4",
    "type_": "Convertible",
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93300,
    "vin": "1234567890"
}
'''

data_json_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': 'Convertible',
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93_300,
    'vin': '1234567890',
    'number_of_doors': 4,
    'registration_country': None,
    'license_plate': None,
}

In [270]:
m = Automobile.model_validate(data)
m

Automobile(manufacturer='BMW', series_name='M4', type_='Convertible', is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, vin='1234567890', number_of_doors=2, registration_country='France', license_plate='AAA-BBB')

In [272]:
m.model_dump() == data_expected_serialization

True

In [273]:
m = Automobile.model_validate_json(data_json)
m

Automobile(manufacturer='BMW', series_name='M4', type_='Convertible', is_electric=False, manufactured_date=datetime.date(2023, 1, 1), base_msrp_usd=93300.0, vin='1234567890', number_of_doors=4, registration_country=None, license_plate=None)

In [274]:
m.model_dump_json() 

'{"manufacturer":"BMW","series_name":"M4","type_":"Convertible","is_electric":false,"manufactured_date":"2023-01-01","base_msrp_usd":93300.0,"vin":"1234567890","number_of_doors":4,"registration_country":null,"license_plate":null}'

In [276]:
m.model_dump()  == data_json_expected_serialization

True

In [277]:
m.model_dump()

{'manufacturer': 'BMW',
 'series_name': 'M4',
 'type_': 'Convertible',
 'is_electric': False,
 'manufactured_date': datetime.date(2023, 1, 1),
 'base_msrp_usd': 93300.0,
 'vin': '1234567890',
 'number_of_doors': 4,
 'registration_country': None,
 'license_plate': None}

#### Modify your Automobile model to implement the following:

- set model to forbid extra fields
- set model to strip whitespace from all string fields
- set model to validate defaults and assignments
- use the enum provided below for the type_ field

In [278]:
class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"

In [280]:
class Automobile(BaseModel):
    model_config = ConfigDict(extra='forbid',
                              str_strip_whitespace=True,
                              validate_assignment=True,
                              validate_default=True)
                              
    manufacturer: str
    series_name: str
    type_: AutomobileType
    is_electric: bool = False
    manufactured_date: date
    base_msrp_usd: float
    vin: str
    number_of_doors: int = 4
    registration_country: str | None = None
    license_plate: str | None = None

Automobile(manufacturer='volvo', 
           series_name='v60', 
           type_= AutomobileType.sedan, 
           manufactured_date=date(2025, 12, 12), 
           base_msrp_usd=499.99, 
           vin='abc123')

Automobile(manufacturer='volvo', series_name='v60', type_=<AutomobileType.sedan: 'Sedan'>, is_electric=False, manufactured_date=datetime.date(2025, 12, 12), base_msrp_usd=499.99, vin='abc123', number_of_doors=4, registration_country=None, license_plate=None)

In [283]:
data_json = '''
{
    "manufacturer": " BMW ",
    "series_name": " M4 ",
    "type_": "Convertible",
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93300,
    "vin": " 1234567890 "
}
'''

data_json_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': AutomobileType.convertible,
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93300.0,
    'vin': '1234567890',
    'number_of_doors': 4,
    'registration_country': None,
    'license_plate': None
}

m = Automobile.model_validate_json(data_json)
m.model_dump() == data_json_expected_serialization

True

In [284]:
m.model_dump() 

{'manufacturer': 'BMW',
 'series_name': 'M4',
 'type_': <AutomobileType.convertible: 'Convertible'>,
 'is_electric': False,
 'manufactured_date': datetime.date(2023, 1, 1),
 'base_msrp_usd': 93300.0,
 'vin': '1234567890',
 'number_of_doors': 4,
 'registration_country': None,
 'license_plate': None}